## Multicast Logistic Regression

The binary classification logistic regression model is a special case of the multicast logistic regression model where the target variable can take more than two values.

For a K-class response variable $y_i$ the multinomial regression model is defined as:

$$
y_i \sim \text{Multinomial}(1, p_{i1}, p_{i2}, \ldots, p_{iK}) \\
\sum_{k = 1}^K p_{ik} = 1 \\
p_{ik} = \frac{\exp(b_k + w_{k1} x_{i1} + \ldots + w_{kp} x_{ip})}{\sum_{j = 1}^K \exp(b_j + w_{j1} x_{i1} + \ldots + w_{jp} x_{ip})}, k = 1, \ldots, K
$$

where the softmax function maps the linear predictor to [0, 1] and ensures that the sum of the probabilities is 1.


In [6]:
#%%
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from sklearn.linear_model import LogisticRegression

np.random.seed(12)

n = 2000
S = 5
K = 3

# generate data
X = np.random.randn(n, S)
beta = np.random.randn(S, K)

linpred = np.dot(X, beta)
prob = np.exp(linpred)
prob /= prob.sum(axis=1)[:, None]

y = np.apply_along_axis(lambda x: np.random.multinomial(1, x).argmax(), axis=1, arr=prob)


In [7]:
beta.T

array([[ 1.34807397,  0.51886255,  0.5165155 ,  1.24904615, -0.6683651 ],
       [ 1.28511998, -1.29529563,  0.45909643,  1.64650635, -1.44853151],
       [ 0.33237059,  0.11223558,  1.02274435, -1.52986962,  0.11771475]])

In [8]:
logreg = LogisticRegression(penalty=None)
logreg.fit(X, y)
logreg.coef_

array([[ 4.47000056e-01,  7.32984167e-01, -1.35208192e-01,
         8.18585767e-01, -1.61549481e-03],
       [ 2.83915372e-01, -1.19124879e+00, -1.87744273e-01,
         1.23329694e+00, -8.36730481e-01],
       [-7.30915428e-01,  4.58264622e-01,  3.22952465e-01,
        -2.05188271e+00,  8.38345975e-01]])

#%% md
The multinomial logistic regression model can be represented as a neural network with a single layer of neurons where each neuron corresponds to a class. The input layer has $p$ neurons, one for each feature, and the output layer has $K$ neurons, one for each class. The weights of the model are represented by the edges connecting the input layer to the output layer (the bias terms are not shown in the diagram). The output of each neuron in the output layer is passed through the softmax function to obtain the predicted probabilities.


```{mermaid}
%%| label: fig-single-neuron-multiclass
%%| fig-width: 6
%%| fig-cap: "ANN model for logistic regression for a single observation"

graph LR
    x1["$$x_{i1}$$"] -->|$$w_1$$| B1(("$$w_{1}^T x_i + b_1$$"))
    x2["$$x_{i2}$$"] -->|$$w_2$$| B2(("$$w_{2}^T x_i + b_2$$"))
    xp["$$x_{ip}$$"] -->|$$w_p$$| B3(("$$w_{p}^T x_i + b_p$$"))
    x1 --> B2
    x1 --> B3
    x2 --> B1
    x2 --> B3
    xp --> B1
    xp --> B2
    B1 --> P1["$$\hat{y}$$"]
    B2 --> P2["$$\hat{y}$$"]
    B3 --> P3["$$\hat{y}$$"]
```

# Entropy and Cross-Entropy

In evaluating a model's accuracy, we need a measure between our model's prediction and a perfect (out-of-sample) prediction. This measure should be able to account for the fact that some outcomes (targets) are easier to predict than others. Consider the task of predicting the weather (sunshine/rain) in a deser, where it almost never rains. A model that always predicts sunshine will be correct most of the time, but it is not a very useful model as you will always be surprised when it rains.

The *entropy* of a distribution is a measure of its uncertainty that has four properties

- It is zero if the distribution is degenerate (i.e. the outcome is always sunshine)
- It is continuous, so a small change in the distribution will result in a small change in the entropy
- It is higher for distributions with can produce more different outcomes than for distributions that can produce fewer outcomes
- It is additive, so the entropy of a distribution is the sum of the entropies of its components. This means that if we first measure the uncertainty about being male/female and then measure the uncertainty about being a soccer fan or not, the uncertainty of the combinations (male/soccer fan, male/not soccer fan, female/soccer-fan, female/not soccer fan) should the sum of the two uncertainties.

It is easy to show that the entropy defined as the expected value of the log-probabilities of the outcomes satisfies these four properties.

$$
H(p) = \sum_{k} p_k \log p_k
$$

So the entropy gives us the uncertainty when predicting outcomes using the true distribution. In classification problems, however, we don't know this distribution. Instead, we rely on a model to produce probabilities that we hope are close to the true probabilities. We can ask: how much does the uncertainty increase if we use the wrong (the model's) probabilities (Q) instead of the true probabilities? This is the *cross-entropy*.

$$
H(P, Q) = H(p) + \text{KL}(p, q)
$$

In the above expression, H(p) is the entropy of the data-generating distribution, and KL(p, q) is the Kullback-Leibler divergence between the data-generating distribution and the model distribution. The KL divergence is always non-negative, and it is zero if the two distributions are identical. Therefore, the cross-entropy is always greater than or equal to the entropy of the data-generating distribution.

$$
\text{KL} = \sum_{k} p_k (\log p_k - \log q_k) = \sum_{i} p_k \log \frac{p_k}{q_k}
$$

The KL-divergence describes how different P and Q are on average (in units of entropy). You have likely encountered a scaled version of it when studying generalized linear models (GLM) under the name of *deviance*. The deviance is the KL-divergence between the data-generating distribution and the model distribution, scaled by a factor of two.  


The loss function for the multinomial logistic regression model is the cross-entropy loss function. The cross-entropy loss function (for a single observation) is defined as:

$$
\text{CE}(y, \hat{y}) = -\sum_{k = 1}^K y_k \log(\hat{y}_k)
$$

where $y$ is the one-hot encoded target variable and $\hat{y}$ is the predicted probability vector. The cross-entropy loss function is minimized by adjusting the weights of the model using gradient descent.

The input layer is mapped to the output layer using the following equations:

$$
z_{ik} = b_k + w_{k1} x_{i1} + \ldots + w_{kS} x_{iS} \\
p_{ik} = \frac{\exp(z_{ik})}{\sum_{j = 1}^K \exp(z_{ij})}, k = 1, \ldots, K
$$

In matrix notation

$$
Z = X_{1 \times S} W^T_{S \times K} + B_{K \times 1} \\
P_{K \times S} = \text{softmax}(Z)
$$

where $X$ is the input matrix, $W$ is the weight matrix, $B$ is the bias matrix, $Z$ is the linear predictor matrix, and $P$ is the predicted probability matrix.

The gradient for a weight $w_{jk}$ single observation is given by:

$$
\frac{\partial \text{CE}(y, \hat{y})}{\partial w_{jk}} = x_{ij} (\hat{y}_k - y_k)
$$

Therefore, the gradient for the whole $S \times K$ weight matrix is:

$$
\frac{\partial \text{CE}(y, \hat{y})}{\partial W} = X_{i} \otimes (\hat{y}_{1 \times K} - y_{1 \times K})
$$


As the models grow more complex, it is important to keep track of the dimensions of the matrices and vectors to ensure that the operations are performed correctly.

So let's break down the operations for a single observation with S=3 features (and we will drop the observation index $i$ for simplicity) and two classes $K = 2$.

The input is a vector $x_{1 \times S}$ (XXX, resolve the notation conflict with the matrix $X$):

$$
x = \begin{bmatrix}
x_1 & x_2 & x_3
\end{bmatrix}
$$

The weight matrix is $W_{S \times K}$ and must map the input to the output layer:

$$
W = \begin{bmatrix}
w_{11} & w_{12} \\
w_{21} & w_{22} \\
w_{31} & w_{32}
\end{bmatrix}
$$

Ignoring the bias term, the linear predictor matrix $Z_{1 \times K}$ is given by:

$$
Z = W^T X = 
\begin{bmatrix}
w_{11} & w_{21} & w_{31} \\
w_{12} & w_{22} & w_{32}
\end{bmatrix}
\begin{bmatrix}
x_1 \\
x_2 \\
x_3
\end{bmatrix}
= \begin{bmatrix}
w_{11} x_1 + w_{21} x_2 + w_{31} x_3 \\
w_{12} x_1 + w_{22} x_2 + w_{32} x_3
\end{bmatrix} = 
\begin{bmatrix}
z_{11} \\
z_{12}
\end{bmatrix}
$$

The predictor matrix is passed through the softmax function to obtain the predicted probability matrix $P_{2 \times 1}$:

$$
\hat{y} = \text{softmax}(Z) = \begin{bmatrix}
\frac{\exp(z_{11})}{\exp(z_{11}) + \exp(z_{12})} \\
\frac{\exp(z_{12})}{\exp(z_{11}) + \exp(z_{12})}
\end{bmatrix} = 
\begin{bmatrix}
\hat{y}_{1} \\
\hat{y}_{2}
\end{bmatrix}
$$

The labels are one-hot encoded, so the target vector $y_{1 \times K}$ is:

$$
y = \begin{bmatrix}
y_1 & y_2
\end{bmatrix}
$$

The cross-entropy loss function is:

$$
\text{CE}(y, \hat{y}) = -\sum_{k = 1}^K y_k \log(\hat{y}_k) = -y_1 \log(\hat{y}_1) - y_2 \log(\hat{y}_2)
$$


Now let's look into the derivatives of the loss function with respect to the weights. The weight matrix is $2 \times 3$, 
therefore the gradient matrix is also $2 \times 3$:

$$
\frac{\partial \text{CE}(y, \hat{y})}{\partial W} = \begin{bmatrix}
\frac{\partial \text{CE}(y, \hat{y})}{\partial w_{11}} & \frac{\partial \text{CE}(y, \hat{y})}{\partial w_{12}} & \frac{\partial \text{CE}(y, \hat{y})}{\partial w_{13}} \\
\frac{\partial \text{CE}(y, \hat{y})}{\partial w_{21}} & \frac{\partial \text{CE}(y, \hat{y})}{\partial w_{22}} & \frac{\partial \text{CE}(y, \hat{y})}{\partial w_{23}}
\end{bmatrix}
$$

Let's derive the gradient for the first weight $w_{11}$. Using the chain rule for differentiation we obtain:

$$
\frac{\partial \text{CE}(y, \hat{y})}{\partial w_{11}} = \frac{\partial \text{CE}(y, \hat{y})}{\partial \hat{y}_1} \frac{\partial \hat{y}_1}{\partial z_{11}} \frac{\partial z_{11}}{\partial w_{11}}
$$

The first term is the derivative of the cross-entropy loss function with respect to the predicted probability $\hat{y}_1$:

$$
\frac{\partial \text{CE}(y, \hat{y})}{\partial \hat{y}_1} = - \frac{\partial}{\partial \hat{y}_1} \left( y_1 \log(\hat{y}_1) + y_2 \log(\hat{y}_2) \right) = - \frac{y_1}{\hat{y}_1}
$$

The second term is the derivative of the softmax function with respect to the linear predictor $z_{11}$:

$$
\frac{\partial \hat{y}_1}{\partial z_{11}} = \frac{\partial}{\partial z_{11}} \left( \frac{\exp(z_{11})}{\exp(z_{11}) + \exp(z_{12})} \right) = \hat{y}_1 (1 - \hat{y}_1)
$$

The last term is the simplest one, as $z_{11}$ is a linear function of $w_{11}$:

$$
\frac{\partial z_{11}}{\partial w_{11}} = \frac{\partial}{\partial w_{11}} \left( w_{11} x_1 + w_{21} x_2 + w_{31} x_3 \right) = x_1
$$

Therefore, the gradient for the weight $w_{11}$ is:

$$
\frac{\partial \text{CE}(y, \hat{y})}{\partial w_{11}} = - \frac{y_1}{\hat{y}_1} \hat{y}_1 (1 - \hat{y}_1) x_1 = - y_1 (1 - \hat{y}_1) x_1
$$

It is important to note that this derivative is non-zero only for the class $k$ that the observation belongs to. For the other classes, the derivative is zero. This means that the weight update will only affect the weights of the class that the observation belongs to, i.e. the column of the weight matrix corresponding to the class (XXX).


The derivative for the weight $w_{21}$ is similar to the previous one. First we find the derivative of the cross-entropy loss function with respect to the predicted probability $\hat{y}_2$:

$$
\frac{\partial \text{CE}(y, \hat{y})}{\partial \hat{y}_2} = - \frac{y_2}{\hat{y}_2}
$$

The derivative of the softmax function with respect to the linear predictor $z_{12}$ is:

$$
\frac{\partial \hat{y}_2}{\partial z_{12}} = \frac{\partial}{\partial z_{12}} \left( \frac{\exp(z_{12})}{\exp(z_{11}) + \exp(z_{12})} \right) = - \hat{y}_1 \hat{y}_2
$$

The derivative of the linear predictor $z_{12}$ with respect to the weight $w_{21}$ is:

$$
\frac{\partial z_{12}}{\partial w_{21}} = x_2
$$

Therefore, the gradient for the weight $w_{21}$ is:

$$
\frac{\partial \text{CE}(y, \hat{y})}{\partial w_{21}} = - \frac{y_2}{\hat{y}_2} \hat{y}_1 \hat{y}_2 x_2 = - y_2 \hat{y}_1 x_2
$$


We can generalize the gradient for the whole weight matrix $W$ as:

$$
\frac{\partial \text{CE}(y, \hat{y})}{\partial W} = X^T (\hat{y} - y)
$$


In [13]:
np.random.seed(23)

W = np.random.randn(S, K)

epochsn = 1000
lr = 0.001

losses = np.zeros(epochsn)

def softmax(x):
    return np.exp(x) / np.exp(x).sum(keepdims=True)

for epoch in range(epochsn):
    for i in range(n):
        x = X[i]
        y_ = y[i]
        
        linpred = np.dot(x, W)
        prob = softmax(linpred)
        
        loss = -np.log(prob[y_])
        losses[epoch] += loss
        
        grad = prob.copy()
        grad[y_] -= 1
        
        W -= lr * np.outer(x, grad)
